In this code we create functions to generate matrix with a specific structure. We take inspiration from paper "Analyzing Multi-Head Self-Attention: Specialized Heads Do the Heavy Lifting, the Rest Can Be Pruned" (https://arxiv.org/abs/1905.09418).


In [2]:
import numpy as np
import scipy


In [17]:
def creatediagonal(d): #d is the vector dimension of the matrix
  vect=np.random.rand(d)
  return (np.diag(vect))


def createsparse(d,perc=0.1): #percentage of sparsity
  sparse_matrix=np.random.rand(d[0], d[1])
  random_mask = np.random.rand(d[0], d[1])
  sparse_matrix[random_mask>perc]=0
  return

#here we create a matrix that for which we attend specific other input randomly.
#For every column we have big (>0.5) elements and the others are zero.

#TO BE COMPLETED.

def structured(d,perc=0.1):
  structured_matrix=np.zeros((d[0],d[1]))
  for j in d[1]:
    random_mask=np.random.rand(d[0]) #we create a mask for each column vector
    random_matrix
    structured_matrix[:,j]=


  return structured_matrix

#create a banded matrix

def context_matrix(d,w): #here w stands for window. d[0]=d[1] in this case
  context_matrix=np.zeros((d[0],d[1]))
  for i in range(2*w):

    np.fill_diagonal(context_matrix, np.random.rand(d[0]-(i-w)), i-w)
  return context_matrix



def addnoise(M,noise=0.1):
  n=M.shape[0]
  m=M.shape[1]
  noise_matrix=np.random.randn(n,m)*M.max()*noise
  M_with_noise=M+noise_matrix
  M_with_noise[M_with_noise > 1] = 1
  M_with_noise[M_with_noise < 0] = 0
  return M_with_noise

def random_matrix(d):
  return np.random.rand(d[0],d[1])


